# ODE Integrators II: Implicit and Symplectic Methods

## Convergence of Integrators

When solving ordinary differential equations (ODEs) numerically,
errors inevitably enter through truncation (e.g., approximating
derivatives with finite differences) and round-off (finite precision
arithmetic).
If these errors grow uncontrollably, the computed solution may diverge
from the true behavior, even if the method is very accurate for short
times.

It is useful to distinguish between several related ideas:
* Accuracy:
  How close the numerical solution is to the exact solution at a given
  time step.
* Consistency:
  The numerical method reproduces the original ODE as
  $\Delta t \to 0$.
  Formally, the local truncation error (the error made in a single
  step assuming exact input) should vanish as $\Delta t \to 0$.
* Stability:
  Errors introduced during computation do not grow uncontrollably as
  steps are repeated.
* Convergence:
  The global numerical solution approaches the exact solution as
  $\Delta t \to 0$.

These concepts are related by a central result of numerical analysis:
\begin{align}
  \text{Consistency + Stability } \implies \text{ Convergence.}
\end{align}

This is known as the
[Lax Equivalence Theorem](https://en.wikipedia.org/wiki/Lax_equivalence_theorem).
The theorem is really for linear finite difference methods for partial
different equation, but it is still useful to discuss it in ODE
integrator.
It gives us a practical recipe:
* First check that the method is consistent (usually straightforward).
* Then analyze stability, we usually use the linear test equation:
  \begin{align}
    \frac{dx}{dt} = \lambda x, \quad \lambda \in \mathbb{C}.
  \end{align}
  Its exact solution is $x(t) = x_0 e^{\lambda t}$.
  Applying a numerical method produces an update:
  \begin{align}
    x_{n+1} = R(z) x_n, \quad z = \lambda \Delta t,
  \end{align}
  where $R(z)$ is the amplification factor.
  A method is stable if
  \begin{align}
    |R(z)| \leq 1,
  \end{align}
  so that errors do not amplify step by step.
  The set of $z$ satisfying this condition defines the stability region.
* Convergence then follows automatically.

## Forward Euler Consistency

The Forward Euler method for $dx/dt = f(x)$ is
\begin{align}
  x_{n+1} = x_n + \Delta t f(x_n).
\end{align}
Expanding the true solution with a Taylor series:
\begin{align}
  x(t_{n+1}) = x(t_n + \Delta t) = x(t_n) + \Delta t f(x_n) + \frac{1}{2} \Delta t^2 f'(x_n) + \mathcal{O}(\Delta t^3)
\end{align}

The difference between the true solution and the Forward Euler step is
$\mathcal{O}(\Delta t^2)$.
Thus, the local truncation error is $\mathcal{O}(\Delta t^2)$, meaning
the method is first-order consistent.

Applying the test equation to forward Euler method gives:
\begin{align}
  x_{n+1} = (1 + \lambda \Delta t) x_n,
\end{align}
so the amplification factor is
\begin{align}
  R(z) = 1 + z, \quad z = \lambda \Delta t.
\end{align}

The stability condition requires
\begin{align}
  |1 + z| \leq 1,
\end{align}
which is the interior of the circle centered at $(-1,0)$ with radius 1
in the complex plane.

In [ ]:
import numpy as np

# Define grid in complex plane
Re = np.linspace(-3, 3, 601)
Im = np.linspace(-3, 3, 601)

Re, Im = np.meshgrid(Re, Im)
Z = Re + 1j * Im

In [ ]:
# Forward Euler amplification factor

R_fE = lambda z: abs(1 + z)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

plt.contour (Re, Im, R_fE(Z), levels=[1],   colors=['C0'])
plt.contourf(Re, Im, R_fE(Z), levels=[0,1], colors=['C0'], alpha=0.1)

plt.legend(handles=[
    mpatches.Patch(color='C0', label="Forward Euler Stable Region"),
])

plt.xlabel(r"Re($z$) = Re($\lambda\Delta t$)")
plt.ylabel(r"Im($z$) = Im($\lambda\Delta t$)")
plt.gca().set_aspect('equal')

Combined with stability, forward Euler is first-order convergent,
i.e., the global error scales as $\mathcal{O}(\Delta t)$, within its
stable region.
Outside the stable region, the method diverges.

### Exponential Decay

To illustrate stability and convergence, let's recall the forward
Euler scheme we implemented last time,

In [ ]:
def Euler(f, x, t, dt, n):
    X = [np.array(x)]
    T = [np.array(t)]
    for _ in range(n):
        X.append(X[-1] + dt * f(X[-1]))
        T.append(T[-1] + dt)
    return np.array(X), np.array(T)

We apply it to solve the equation $dx/dt = -x$, which has $\lambda =
-1$ and solution $\exp(-t)$.

In [ ]:
f = lambda x: -x
t = 16
T = np.linspace(0, t, 1001)
X = np.exp(-T)

n = 32; X1, T1 = Euler(f, x=1, t=0, dt=t/n, n=n)
n = 16; X2, T2 = Euler(f, x=1, t=0, dt=t/n, n=n)
n = 8;  X3, T3 = Euler(f, x=1, t=0, dt=t/n, n=n)
n = 4;  X4, T4 = Euler(f, x=1, t=0, dt=t/n, n=n)

plt.plot(T,  X,        label=r"$\exp(t)$")
plt.plot(T1, X1, "o-", label=f"Forward Euler with dt={t/32}")
plt.plot(T2, X2, "o-", label=f"Forward Euler with dt={t/16}")
plt.plot(T3, X3, "o-", label=f"Forward Euler with dt={t/8} (critical stability)")
plt.plot(T4, X4, "o-", label=f"Forward Euler with dt={t/4}")
plt.xlabel(r"$t$")
plt.ylabel(r"$x(t)$")
plt.ylim(-10, 10)
plt.legend()

The error for $\Delta t = 4$ grows unbounded.
Even at the critical stability step size $\Delta t = 2$, the
oscillating behavior is fundamentally wrong.
Nevertheless, as $\Delta t \rightarrow 0$, especially when
$\Delta t < -2/\lambda$ the solution does convergence.

However, if $\lambda$ is positive, there is no positive $\Delta t$
that can make forward Euler stable.
And we call forward Euler unconditionally unstable.

This suggests that,
* Consistency:
  Forward Euler has local error $\mathcal{O}(\Delta t^2)$.
* Stability:
  Stable only inside the unit disk centered at $(-1,0)$.
* Convergence:
  By Lax's theorem, Forward Euler converges with global error
  $\mathcal{O}(\Delta t)$.

Implications:
* For $\lambda < 0$ (decaying solutions), stability requires
  $\Delta t \leq -2/\lambda$.
* For $\lambda > 0$ (growing solutions), Forward Euler is
  unconditionally unstable: no choice of $\Delta t$ can control error
  growth.
* For stiff systems, where $|\lambda|$ is very large, the step size
  restriction is too severe to be practical.

This motivates the need for implicit integrators (Backward Euler,
trapezoidal rule) and later symplectic integrators for Hamiltonian
systems.

### Simple Harmonic Oscillator

The exponential test equation matches the linear test equation and is
a good illustrative example, but many physical systems of interest are
naturally written as systems of coupled ODEs.

Recalling again from last lecture, an important example is the simple
harmonic oscillator:
\begin{align}
  \frac{d}{dt}
  \begin{bmatrix}
    \theta(t) \\
    \Omega(t)
  \end{bmatrix}
  =
  \begin{bmatrix}
    0    & 1 \\
    -g/l & 0
  \end{bmatrix}
  \begin{bmatrix}
    \theta(t) \\
    \Omega(t)
  \end{bmatrix}.
\end{align}
Here $\theta(t)$ is the angular displacement, $\Omega(t)$ the angular
velocity, and $g/l$ sets the natural frequency $\omega^2 = g/l$.


Applying the Forward Euler method gives:
\begin{align}
  \begin{bmatrix}
    \theta_{n+1} \\
    \Omega_{n+1}
  \end{bmatrix}
  =
  \begin{bmatrix}
    \theta_{n} \\
    \Omega_{n}
  \end{bmatrix}
  +
  \Delta t
  \begin{bmatrix}
    0 & 1 \\
    -g/l & 0
  \end{bmatrix}
  \begin{bmatrix}
    \theta_{n} \\
    \Omega_{n}
  \end{bmatrix}.
\end{align}

This can be written compactly as:
\begin{align}
  \mathbf{x}_{n+1} = A \mathbf{x}_n, \quad
  A =
  \begin{bmatrix}
    1              & \Delta t \\
    -(g/l)\Delta t & 1
  \end{bmatrix}.
\end{align}

Here $A$ is the amplification matrix, the multi-dimensional analogue
of the scalar amplification factor $R(z)$.
Stability requires that the eigenvalues of $A$ satisfy
$|\lambda| \leq 1$.

Let's compute them.
The characteristic polynomial of $A$ is:
\begin{align}
  \det(A - \lambda I) =
  \begin{vmatrix}
  1 - \lambda    & \Delta t \\
  -(g/l)\Delta t & 1 - \lambda
  \end{vmatrix}
  = (1-\lambda)^2 + \frac{g}{l} \Delta t^2.
\end{align}

Setting this to zero,  $(1 - \lambda)^2 = -(g/l)\Delta t^2$.
Thus the eigenvalues are:
\begin{align}
  \lambda_{\pm} = 1 \pm i \sqrt{\frac{g}{l}} \Delta t.
\end{align}

The magnitude of each eigenvalue is:
\begin{align}
  |\lambda_{\pm}| = \sqrt{1 + \frac{g}{l} \Delta t^2}.
\end{align}
This is always larger than 1 for any nonzero time step.
Therefore, Forward Euler applied to the harmonic oscillator is
unconditionally unstable.

In [ ]:
# Parameters
g, l = 9.8, 1
t, n = 100, 10000

def F(x):
    theta, omega = x
    return np.array([omega, -(g/l) * theta])

T     = np.linspace(0, t, 1001)
Theta = np.cos(np.sqrt(g/l)*T)

X1, T1 = Euler(F, x=[1.0, 0.0], t=0, dt=t/n, n=n)
Theta1, Omega1 = X1[:,0], X1[:,1]

plt.plot(T,  Theta, '--', label=r"Exact $\theta(t)$")
plt.plot(T1, Theta1,      label=r"Euler $\theta(t)$")
plt.xlabel(r"$t$")
plt.ylabel(r"$\theta(t)$")
plt.ylim(-10, 10)
plt.legend()

In [ ]:
# HANDSON: try increase the number of time step per period.
#          Does this stop the oscillator from blowing up?


The true solution of the harmonic oscillator is bounded and
oscillatory.
However, the forward Euler discretization produces eigenvalues with
modulus larger than 1.
This means that the numerical solution grows exponentially in
amplitude while also oscillating.
This "blow up" happens even for very small $\Delta t$.

In [ ]:
# HANDSON: try plotting the energy of the oscillator.


For the harmonic oscillator, Forward Euler is inconsistent with the
bounded, energy-conserving nature of the physics.
In fact,
* Explicit (forward)  Euler introduces artificial growth  of energy.
* Implicit (backward) Euler introduces artificial damping of energy.
* Symplectic integrators (e.g., Leapfrog, see below) preserve the
  energy structure and produce qualitatively correct long-term
  behavior.

This example connects stability analysis directly to physics:
the choice of integrator is not just about error size, but about
preserving the qualitative features of the dynamics.

## Stability Regions for RK2 and RK4

So far we have analyzed the forward Euler method, which is simple but
has a very restrictive stability region.
In practice, physicists often use higher-order explicit Runge-Kutta
(RK) methods, such as RK2 and RK4.
These methods improve accuracy while also enlarging the stability
region, allowing for larger time steps in many problems.

To analyze stability, we again apply the methods to the linear test equation:
\begin{align}
  \frac{dx}{dt} = \lambda x, \quad z = \lambda \Delta t.
\end{align}
Each Runge-Kutta scheme can be written in the form:
\begin{align}
  x_{n+1} = R(z) x_n,
\end{align}
where $R(z)$ is the stability function of the method.
Stability requires $|R(z)| \leq 1$.
* Forward Euler (for comparison):
  \begin{align}
    R_{\text{fE}}(z) = 1 + z
  \end{align}
* RK2 (Heun's method):
  \begin{align}
    R_{\text{RK2}}(z) = 1 + z + \tfrac{1}{2} z^2
  \end{align}
* RK4 (classical Runge–Kutta):
  \begin{align}
    R_{\text{RK4}}(z) = 1 + z + \tfrac{1}{2} z^2 + \tfrac{1}{6} z^3 + \tfrac{1}{24} z^4
  \end{align}

Notice:
$R_{\text{RK4}}(z)$ is simply the first four terms of the Taylor
expansion of $e^z$, which explains why RK4 is accurate to fourth
order.

In [ ]:
# Stability functions
R_fE  = lambda z: abs(1 + z)
R_RK2 = lambda z: abs(1 + z + 0.5 * z**2)
R_RK4 = lambda z: abs(1 + z + 0.5 * z**2 + (1/6) * z**3 + (1/24) * z**4)

# Plot regions
methods = {
    'Forward Euler': (R_fE,  'C0'),
    'RK2':           (R_RK2, 'C1'),
    'RK4':           (R_RK4, 'C2'),
}

for name, (R, color) in methods.items():
    plt.contour (Re, Im, R(Z), levels=[1],   colors=[color])
    plt.contourf(Re, Im, R(Z), levels=[0,1], colors=[color], alpha=0.1)

plt.legend(handles=[
    mpatches.Patch(color=color, label=name) for name, (_,color) in methods.items()
])

plt.xlabel(r"Re($\lambda \Delta t$)")
plt.ylabel(r"Im($\lambda \Delta t$)")
plt.gca().set_aspect('equal')